In [1]:
# globally useful imports of standard libraries needed in this notebook
import numpy as np
import pandas as pd

import random

# import project specific modules used in this notebook
import sys
sys.path.append('../src')
import mindwandering.data

# Mindwandering Feature Selection

In the reference paper being replicated, feature selection was applied, and the top 25%, 50% or 75% of the top-ranked
features were used to build models.

The paper describes using a correlation-based feature selection algorithm from WEKA.  It is descriped thus:

```
To avoid overfitting, feature selection was performed on a random 66% of the participants in the
training set.  The process was repeated five times to ameliorate variance called by random selection
of participants.  The feature rankings were then averaged over these five iterations.
```

```
Features were ranked higher if they were weakly correlated with other features but strongly correlated with
mind-wandering reports using a correlation-based feature selection algorithm from WEKA
```

In this notebook we will explore this and possible some other feature selection methods.  I am not sure if we
should try and exactly replicate this described method, I suppose if we think it is necessary we can follow the
reference, and maybe even see if there is a python library implementation purporting to be this type
of feature selection.

For feature selection we need the features, of course.  We will try using the unscaled features, though it shouldn't matter for this feature
selction if they are scaled or not.

Actually lets also get the standard scaled features, and see if we get any significantly different features selected
using scaled features.  Maybe it could be significant if using an estimator that is sensitive to unscaled features?

We also need the `mind_wandered_label` so we can calculate correlation of features with the label.

And we actually need the `participant_id` information from the experiment metadata so that we can select samples
based on participant.

In [2]:
# We will perform feature selection on the unscaled feature data in this section.
df_features = mindwandering.data.get_df_features()

# and actually, lets try feature selection with the standard scaled features as well
df_features_standard_scaled = mindwandering.data.transform_df_features_standard_scaled(df_features)

# We also need the label data to correlate features with the mind_wandered_label
# we only need the binary mind_wandered_label for this 
df_label = mindwandering.data.get_df_label()
df_label = df_label.mind_wandered_label.copy()

# And we need the experiment metadata so that we can select samples based on participant id
df_experiment_metadata = mindwandering.data.get_df_experiment_metadata()

# Hand-Made Replication of Described Paper Feature Selection using Feature Correlations

In this section we attempt to replicate the general approach described in the paper used for feature selection.

We need the features, of course.  We will try using the unscaled features, though it shouldn't matter for this feature
selction if they are scaled or not.

We also need the `mind_wandered_label` so we can calculate correlation of features with the label.

And we actually need the `participant_id` information from the experiment metadata so that we can select samples
based on participant.

Lets make a function to select a random sample of some percentage of participants.

In [3]:
def get_random_sample_of_participants(participant_ids, sample_ratio = 0.66):
    """We expect a list or series of participant ids.  This method selects
    the sample_size proportion of the participants at random, and returns this
    sampled list.
    """
    num_participants = len(participant_ids)
    num_samples = int(num_participants * sample_ratio)
    return random.sample(participant_ids, num_samples)

In [4]:
# test we get a random sample.  There should be 135 participants, and 0.66 * 135 = 89 sampled
participant_ids = df_experiment_metadata.participant_id.unique().tolist()
participant_sample = get_random_sample_of_participants(participant_ids, 0.02)
print('Number of participants sampled: ', len(participant_sample))

# now test we can get this sample from dataframe
idx = df_experiment_metadata.participant_id.isin(participant_sample)
print('Number of indexes pulled in sample (will not equal number of participants): ', sum(idx))
print('Participants sampled: ', participant_sample)

Number of participants sampled:  2
Number of indexes pulled in sample (will not equal number of participants):  58
Participants sampled:  ['1096-ND', '1043-UM']


In [5]:
# visually test/confirm that only trials of sampled participants are in the sample
# basically ensureing same table indexes have been selected in the feature sample
trials = df_features[idx]
trials

,fixation_duration_median,fixation_duration_mean,fixation_duration_standard_deviation,fixation_duration_minimum,fixation_duration_maximum,fixation_duration_range,fixation_duration_skew,fixation_duration_kurtosis,saccade_duration_median,saccade_duration_mean,...,pupil_diameter_maximum,pupil_diameter_range,pupil_diameter_skew,pupil_diameter_kurtosis,number_of_blinks,blink_duration_mean,number_of_saccades,horizontal_saccade_proportion,fixation_dispersion,fixation_saccade_durtion_ratio
2069,249.5,272.500000,111.588530,133.0,433.0,300.000000,0.245799,-1.408472,34.000000,193.000000,...,0.865165,1.953822,-0.087945,-1.488123,0,0.0,7.0,1.000000,0.502,1.614000
2070,283.0,268.500000,66.723309,167.0,366.0,199.000000,-0.385774,-0.462146,34.000000,121.428571,...,-1.784033,0.301973,-0.441956,-0.479486,0,0.0,7.0,0.857143,0.394,2.527000
2071,267.0,289.090909,91.478363,150.0,449.0,299.000000,0.402579,-0.496922,33.500000,56.700000,...,-0.048682,1.944449,0.575628,-0.587321,0,0.0,10.0,1.000000,0.592,5.608000
2072,283.0,323.888889,159.865134,117.0,583.0,466.000000,0.633635,-0.932145,25.000000,56.125000,...,0.519180,0.795864,-0.016398,-1.262740,2,200.0,8.0,1.000000,0.422,6.492000
2073,200.0,190.769231,57.931215,116.0,300.0,184.000000,0.372001,-0.755695,25.000000,82.000000,...,-1.984771,0.525766,0.639939,0.978010,0,0.0,12.0,0.916667,0.441,2.520000
2074,292.0,278.200000,89.086475,100.0,383.0,283.000000,-1.144553,0.695938,17.000000,31.444444,...,-0.434311,0.744930,0.906469,1.085498,0,0.0,9.0,1.000000,0.442,9.830000
2075,367.0,376.142857,160.280409,150.0,667.0,517.000000,0.685067,1.635405,66.500000,121.833333,...,1.039063,0.955997,-0.368558,-0.778355,1,233.0,6.0,1.000000,0.556,3.602000
2076,217.0,266.545455,125.415600,134.0,533.0,399.000000,1.123214,0.636508,33.000000,46.600000,...,-0.229395,1.399166,0.225565,-1.730487,0,0.0,10.0,1.000000,0.474,6.292000
2077,249.0,292.090909,146.603857,83.0,566.0,483.000000,0.438095,-0.400557,17.000000,70.000000,...,1.845951,1.215578,0.783545,-0.081540,1,149.0,10.0,1.000000,0.482,4.590000
2078,200.0,204.714286,94.760775,84.0,383.0,299.000000,0.374160,-0.952404,33.000000,65.230769,...,0.194233,0.781913,1.408791,1.447875,2,192.0,13.0,1.000000,0.426,3.380000


In [6]:
# visually test/confirm that only trials of sampled participants are in the sample
participants = df_experiment_metadata.participant_id[idx]
participants

2069    1043-UM
2070    1043-UM
2071    1043-UM
2072    1043-UM
2073    1043-UM
2074    1043-UM
2075    1043-UM
2076    1043-UM
2077    1043-UM
2078    1043-UM
2079    1043-UM
2080    1043-UM
2081    1043-UM
2082    1043-UM
2083    1043-UM
2084    1043-UM
2085    1043-UM
2086    1043-UM
2087    1043-UM
2088    1043-UM
2089    1043-UM
2090    1043-UM
2091    1043-UM
2092    1043-UM
2093    1043-UM
2094    1043-UM
2095    1043-UM
2096    1043-UM
2097    1043-UM
3767    1096-ND
3768    1096-ND
3769    1096-ND
3770    1096-ND
3771    1096-ND
3772    1096-ND
3773    1096-ND
3774    1096-ND
3775    1096-ND
3776    1096-ND
3777    1096-ND
3778    1096-ND
3779    1096-ND
3780    1096-ND
3781    1096-ND
3782    1096-ND
3783    1096-ND
3784    1096-ND
3785    1096-ND
3786    1096-ND
3787    1096-ND
3788    1096-ND
3789    1096-ND
3790    1096-ND
3791    1096-ND
3792    1096-ND
3793    1096-ND
3794    1096-ND
3795    1096-ND
Name: participant_id, dtype: object

Correlation of each feature with the label is relatively easy.  Lets use standard pearson correlation to calcualte correlation of
each feature in sample with the `mind_wandered_label`

Correlation of each feature ends up in a series, with the index being the feature label.  This is using the
default pearson correlation.  Results can range from 1.0 for perfectly correlated to -1.0 for uncorrelated.

In [7]:
participant_ids = df_experiment_metadata.participant_id.unique().tolist()
participant_sample = get_random_sample_of_participants(participant_ids, 0.66)
idx = df_experiment_metadata.participant_id.isin(participant_sample)
feature_sample = df_features[idx]
label_sample = df_label[idx]

print(feature_sample.shape)
print(label_sample.shape)

label_corr = feature_sample.corrwith(label_sample)
label_corr.describe()

(2603, 62)
(2603,)


count    62.000000
mean      0.004717
std       0.060884
min      -0.109313
25%      -0.032351
50%       0.000497
75%       0.030618
max       0.169791
dtype: float64

It is also possible to calculate the pairwise correlations of a pandas dataframe with a single method.  Here is what you get when doin that.

In [8]:
feature_sample.corr()

,fixation_duration_median,fixation_duration_mean,fixation_duration_standard_deviation,fixation_duration_minimum,fixation_duration_maximum,fixation_duration_range,fixation_duration_skew,fixation_duration_kurtosis,saccade_duration_median,saccade_duration_mean,...,pupil_diameter_maximum,pupil_diameter_range,pupil_diameter_skew,pupil_diameter_kurtosis,number_of_blinks,blink_duration_mean,number_of_saccades,horizontal_saccade_proportion,fixation_dispersion,fixation_saccade_durtion_ratio
fixation_duration_median,1.000000,0.817753,0.413990,0.437275,0.411862,0.369824,-0.190745,-0.142826,-0.040159,-0.090129,...,-0.033711,-0.095368,0.001211,-0.052644,-0.045993,-0.054175,-0.457368,0.032485,0.026302,0.343638
fixation_duration_mean,0.817753,1.000000,0.801724,0.450992,0.782075,0.737182,0.131896,0.049816,-0.029900,-0.084378,...,-0.043392,-0.106463,-0.005387,-0.052547,-0.035493,-0.050002,-0.559870,-0.008604,0.015799,0.418473
fixation_duration_standard_deviation,0.413990,0.801724,1.000000,0.096217,0.963444,0.955718,0.441343,0.338495,0.062548,0.028372,...,-0.046024,-0.090698,-0.021364,-0.031870,0.009061,-0.001850,-0.508644,-0.094041,0.011765,0.266379
fixation_duration_minimum,0.437275,0.450992,0.096217,1.000000,0.185988,0.044108,0.130029,0.030280,-0.126812,-0.138376,...,-0.000784,-0.021852,0.008881,-0.009083,-0.114024,-0.105350,-0.219398,0.115720,0.035433,0.317425
fixation_duration_maximum,0.411862,0.782075,0.963444,0.185988,1.000000,0.981208,0.587637,0.514827,-0.013755,-0.068803,...,-0.033199,-0.077052,-0.020129,-0.051065,-0.009117,-0.018480,-0.415987,-0.043977,-0.004383,0.333655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
blink_duration_mean,-0.054175,-0.050002,-0.001850,-0.105350,-0.018480,-0.004384,-0.014830,-0.022727,0.000620,0.082974,...,0.075792,-0.023968,0.022680,0.019999,0.731707,1.000000,-0.055383,-0.012285,-0.047311,-0.270292
number_of_saccades,-0.457368,-0.559870,-0.508644,-0.219398,-0.415987,-0.398022,0.011313,0.047066,-0.396392,-0.544421,...,0.060563,0.138427,-0.008162,-0.096196,-0.018358,-0.055383,1.000000,0.212173,-0.114678,0.150556
horizontal_saccade_proportion,0.032485,-0.008604,-0.094041,0.115720,-0.043977,-0.064646,0.006131,0.007165,-0.250041,-0.257436,...,-0.067479,-0.074956,0.032497,-0.055030,-0.000400,-0.012285,0.212173,1.000000,0.008332,0.204544
fixation_dispersion,0.026302,0.015799,0.011765,0.035433,-0.004383,-0.015379,-0.034999,-0.027951,0.104956,0.104153,...,0.016368,0.030054,-0.042863,-0.000427,-0.048142,-0.047311,-0.114678,0.008332,1.000000,-0.067946


I'm not sure what is considered the standard approach here.  I'm inclined to just take the maximum, excluding the self correlation, and
using that as a proxy measure for how correlated a feature is with another feature.

I would like to end up with the same type of series, with 1 row for each that has the maximum correlation, and the feature labels as the
indexes.

In [9]:
# convert to numpy array, so we can make the diagnol -1 and easily calculate maximum
feature_corr = feature_sample.corr().to_numpy()
np.fill_diagonal(feature_corr, -1)
feature_corr = feature_corr.max(axis=1)

Lets write a function then to score correlation as a combination of the correlation with the label, and with the other features.

- First of all, we'll take the absolute value, as a big negative correlation is still a high correlation.
- Then since high correlation with label is good, but want low correlation with other features, we'll invert the feature correlation by
  subtracting it from 1.
- Then we just combine the two scores.  I'll make it a weighted average that we could adjust to favor label or feature correlation,
  but we'll try weight of 50/50 to begin with.

In [10]:
def calculate_feature_correlation_scores(df_features, df_label, label_weight=0.5):
    """This function will generate a correlation score for a dataframe of features and a series of the
    target labels.  The score is a combination of how correlated each feature is with the label, and how
    correlated each feature is with other features.  
    
    We calculate all correlations using standard pearson correlation.  The correlation of each feature with
    the output label is a single score.  We take the absolute value of this correlation, as it is the
    magnitude of the correlation that is of interest.  We do a pairwise correlation of each feature with 
    all other features, and then determine the maximum correlation (that is not self-correlation).
    Again we use the absolute value of this maximum correlation.  However, while high correlation with
    the label is good, high correlation with other features is bad.  So we invert the correlation score
    with other features (1 - feature_correlation).  Then we take the weighted average of the label
    correlation and feature correlation.
    
    The result is a single score between 0 and 1 for each feature.  We return this result as a pandas
    series, with the feature labels as index (in original order given in input dataframe), and their
    calculated combined feature correlation score as the value.
    """
    # first calculate correlation score of features with the label, convert to numpy array, and take
    # absolute value.  Result is a vector of the absolute value of the correlation with the label
    label_corr = df_features.corrwith(df_label).fillna(0)
    label_corr = label_corr.to_numpy()
    label_corr = np.absolute(label_corr)
    
    # now calculate cross correlation of each feature with all the others.  Result is an nxn square
    # array of correlation values.  Values in diagnol will be 1 because of self-correlation.
    # replace diagnol by -1 and take the max of each row to get maximum correlation of each feature.
    # Take absolute value before max, as we want the magnitude of the correlation
    feature_corr = df_features.corr().fillna(0).to_numpy()
    feature_corr = np.absolute(feature_corr)
    np.fill_diagonal(feature_corr, -1)
    feature_corr = feature_corr.max(axis=1)
    
    # now combine correlations using a weighted average
    combined_corr = (label_weight * label_corr) + ((1.0 - label_weight) * feature_corr)
    
    # now convert back to a pandas series
    combined_corr_series = pd.Series(data=combined_corr, index=df_features.columns)
    
    return combined_corr_series

In [11]:
# test the function
participant_ids = df_experiment_metadata.participant_id.unique().tolist()
participant_sample = get_random_sample_of_participants(participant_ids, 0.66)
idx = df_experiment_metadata.participant_id.isin(participant_sample)
feature_sample = df_features[idx]
label_sample = df_label[idx]

corr_scores = calculate_feature_correlation_scores(feature_sample, label_sample)

In [12]:
corr_scores

fixation_duration_median                0.439179
fixation_duration_mean                  0.453311
fixation_duration_standard_deviation    0.513725
fixation_duration_minimum               0.272946
fixation_duration_maximum               0.520856
                                          ...   
blink_duration_mean                     0.370127
number_of_saccades                      0.303304
horizontal_saccade_proportion           0.286517
fixation_dispersion                     0.182719
fixation_saccade_durtion_ratio          0.399315
Length: 62, dtype: float64

We should be able to sort this series by the feature correlation scores

In [13]:
corr_scores.sort_values(ascending=False)

pupil_diameter_mean          0.576633
pupil_diameter_median        0.575534
saccade_amplitude_range      0.556825
saccade_amplitude_maximum    0.555569
fixation_duration_maximum    0.520856
                               ...   
saccade_amplitude_minimum    0.201225
fixation_dispersion          0.182719
pupil_diameter_kurtosis      0.124682
saccade_velocity_kurtosis    0.114180
pupil_diameter_skew          0.105354
Length: 62, dtype: float64

Putting this togeter, we want to then draw N=5 samples, compute the feature correlation scores of each sample, averge the
scores for each feature, and then sort the features by the scores.  This sorted result could then be used to 
select some percentage of the highest scored features.

In [14]:
def rank_features_using_correlation_scores(df_features, df_label, participant_ids, N=5, label_weight=0.5, sample_ratio=0.66):
    """Perform the complete procedure to rank features using some type of correlation score measure.  We assume
    a function that will calculate correlation scores of features, given a dataframe of the features and a dataframe
    of the labels.
    
    This method performs N=5 (by default) samples of the features.  Samples are drawn by subject, where 66% of
    subjects are sampled each trial.  We combine the calculated feature scores by taking the average.
    We then rank the features (sort them), and return this resulting series of ranked features.
    """
    # a series to accumulate and calculate the final result to return
    num_trials, num_features = df_features.shape
    corr_scores_result = pd.DataFrame(data=np.zeros((num_features,N)), index=df_features.columns)

    # get list of participant ids to use in sampling loop
    #participant_ids = df_experiment_metadata.participant_id.unique().tolist()
    
    for sample_num in range(N):
        # draw a sample of participants
        participant_sample = get_random_sample_of_participants(participant_ids, sample_ratio)
        idx = df_experiment_metadata.participant_id.isin(participant_sample)
        feature_sample = df_features[idx]
        label_sample = df_label[idx]
        
        # calculate the feature correlation scores for this randomly drawn sample
        corr_scores = calculate_feature_correlation_scores(feature_sample, label_sample)
        
        # add the scores to the result
        corr_scores_result.iloc[:,sample_num] = corr_scores
        
    # add new column of the average feature correlation scores
    corr_scores_result['mean'] = corr_scores_result.mean(axis=1)
        
    # sort by the mean to get final ranking
    corr_scores_result = corr_scores_result.sort_values(by=['mean'], ascending=False)
    
    return corr_scores_result

## Unscaled Feature Selection

Using the general functions developed above, lets use them to show example of feature selection using
this method, using the general unscaled set of features.

In [15]:
# test the feature ranking
participant_ids = df_experiment_metadata.participant_id.unique().tolist()
corr_scores = rank_features_using_correlation_scores(df_features, df_label, participant_ids, label_weight=0.5)
corr_scores

,0,1,2,3,4,mean
pupil_diameter_median,0.558273,0.567535,0.580534,0.560917,0.568417,0.567135
pupil_diameter_mean,0.558539,0.567235,0.581149,0.560725,0.567727,0.567075
saccade_amplitude_range,0.554393,0.549001,0.563638,0.559019,0.566948,0.558600
saccade_amplitude_maximum,0.553630,0.547327,0.562808,0.559031,0.568058,0.558171
fixation_duration_maximum,0.519933,0.531438,0.528331,0.521639,0.506540,0.521576
...,...,...,...,...,...,...
saccade_amplitude_minimum,0.191302,0.197346,0.189196,0.206294,0.197849,0.196397
fixation_dispersion,0.192961,0.183615,0.188847,0.194630,0.189883,0.189987
pupil_diameter_kurtosis,0.127061,0.128688,0.133435,0.131552,0.133573,0.130862
saccade_velocity_kurtosis,0.121666,0.127119,0.113555,0.131037,0.134055,0.125487


This seems to be working as expected.  Though I do notice from running that given equal weight between label and
cross-feature correlation, the first 4 ranked features tendto be pupil_diameter measures.  I would have thought these
would all be correlated with one another.  Might want to increase weight for the penality for features being correlated
to see if that gets different results?

Give this, we can just take the desired X% of features, and create a df_features dataframe with only those features.
For example, if we want the top 10% of features, where we give label correlation a weight of 0.25, we get:

In [16]:
corr_scores = rank_features_using_correlation_scores(df_features, df_label, participant_ids, label_weight=0.25)
print(corr_scores.iloc[:10,5])

best_features = corr_scores.iloc[:10].index.tolist()
print(best_features)

pupil_diameter_median                   0.566062
pupil_diameter_mean                     0.565246
saccade_amplitude_range                 0.558937
saccade_amplitude_maximum               0.558884
fixation_duration_maximum               0.524048
fixation_duration_range                 0.521302
fixation_duration_standard_deviation    0.516630
saccade_duration_maximum                0.503972
saccade_amplitude_standard_deviation    0.503273
saccade_duration_range                  0.502171
Name: mean, dtype: float64
['pupil_diameter_median', 'pupil_diameter_mean', 'saccade_amplitude_range', 'saccade_amplitude_maximum', 'fixation_duration_maximum', 'fixation_duration_range', 'fixation_duration_standard_deviation', 'saccade_duration_maximum', 'saccade_amplitude_standard_deviation', 'saccade_duration_range']


In [17]:
df_features[best_features]

,pupil_diameter_median,pupil_diameter_mean,saccade_amplitude_range,saccade_amplitude_maximum,fixation_duration_maximum,fixation_duration_range,fixation_duration_standard_deviation,saccade_duration_maximum,saccade_amplitude_standard_deviation,saccade_duration_range
1,-1.362156,-1.458751,713.032333,793.888635,366.0,283.0,101.294620,450.0,210.956418,434.0
2,-0.200080,-0.211293,859.676639,926.600337,499.0,366.0,107.757556,383.0,289.943023,367.0
3,0.992546,0.974411,821.302475,887.889747,416.0,333.0,100.261005,466.0,288.874145,450.0
4,-0.877777,-0.860678,784.969112,801.902460,516.0,433.0,125.994157,250.0,201.615678,234.0
5,0.848486,0.866260,730.301436,799.075150,250.0,134.0,41.252732,399.0,270.211194,383.0
...,...,...,...,...,...,...,...,...,...,...
4072,0.794009,0.879699,580.088059,586.295965,350.0,208.0,64.849056,242.0,147.530726,234.0
4073,0.186908,0.133948,812.749371,887.082673,425.0,300.0,76.666812,158.0,257.982901,150.0
4074,0.651010,0.633963,554.076217,671.091967,642.0,551.0,141.660580,159.0,161.038748,151.0
4075,0.424620,0.424035,750.842720,814.580633,300.0,208.0,53.755369,250.0,218.216709,242.0


## Standard Scaled Feature Selection

And lets test the assumption that at least for this method scaling of features shouldn't effect the results.
Rerun the feature selection using the standard scaled set of features.

In [18]:
# test the feature ranking
corr_scores = rank_features_using_correlation_scores(df_features_standard_scaled, df_label, participant_ids, label_weight=0.5)
corr_scores

,0,1,2,3,4,mean
pupil_diameter_median,0.562343,0.586882,0.551453,0.581862,0.572264,0.570961
pupil_diameter_mean,0.560811,0.586918,0.551434,0.581009,0.570501,0.570135
saccade_amplitude_range,0.551620,0.556492,0.540441,0.563497,0.568965,0.556203
saccade_amplitude_maximum,0.550986,0.556942,0.540149,0.562388,0.569833,0.556060
fixation_duration_maximum,0.510140,0.536150,0.514278,0.530942,0.522177,0.522737
...,...,...,...,...,...,...
saccade_amplitude_minimum,0.202874,0.189402,0.199999,0.203308,0.197629,0.198642
fixation_dispersion,0.188826,0.190920,0.201640,0.193844,0.198948,0.194836
pupil_diameter_kurtosis,0.133188,0.129343,0.141532,0.131864,0.144813,0.136148
saccade_velocity_kurtosis,0.123577,0.115425,0.128979,0.133678,0.121256,0.124583


In [19]:
corr_scores = rank_features_using_correlation_scores(df_features_standard_scaled, df_label, participant_ids, label_weight=0.25)
print(corr_scores.iloc[:10,5])

best_features = corr_scores.iloc[:10].index.tolist()
print(best_features)

pupil_diameter_median                   0.573013
pupil_diameter_mean                     0.572087
saccade_amplitude_range                 0.550405
saccade_amplitude_maximum               0.550304
fixation_duration_maximum               0.516072
fixation_duration_range                 0.514160
fixation_duration_standard_deviation    0.509663
pupil_diameter_maximum                  0.506191
pupil_diameter_minimum                  0.505064
saccade_angle_relative_mean             0.502239
Name: mean, dtype: float64
['pupil_diameter_median', 'pupil_diameter_mean', 'saccade_amplitude_range', 'saccade_amplitude_maximum', 'fixation_duration_maximum', 'fixation_duration_range', 'fixation_duration_standard_deviation', 'pupil_diameter_maximum', 'pupil_diameter_minimum', 'saccade_angle_relative_mean']


In this case, some small differences in the median feature rank score, and some slightly different order,
but at least we get the same first 10 in the same order.
But we are doing sampling, so we should certainly expect differences in the feature rank scores here which might
cause small rearrangements in order.  We could try and increase the number of samples N which might make
results converge completely.

## Scaled and Outliers Removed

Had some problems with data frame with outliers removed.  Try it with that dataframe.

In [20]:
df_features_outliers_removed = mindwandering.data.transform_df_features_outliers_removed(df_features_standard_scaled)

In [21]:
corr_scores = rank_features_using_correlation_scores(df_features_outliers_removed, df_label, participant_ids, label_weight=0.5)
pd.set_option('display.max_rows', 500)
corr_scores

,0,1,2,3,4,mean
pupil_diameter_median,0.583628,0.601705,0.574824,0.567964,0.570336,0.579691
pupil_diameter_mean,0.584064,0.600342,0.573328,0.566692,0.571317,0.579149
saccade_amplitude_maximum,0.532431,0.550198,0.547880,0.549653,0.546691,0.545371
saccade_amplitude_range,0.532719,0.549977,0.548035,0.548458,0.545236,0.544885
pupil_diameter_maximum,0.524717,0.533594,0.517252,0.513581,0.512990,0.520427
fixation_duration_maximum,0.517983,0.497104,0.522954,0.525956,0.516274,0.516054
fixation_duration_range,0.514211,0.493346,0.520456,0.522978,0.512376,0.512673
pupil_diameter_minimum,0.518862,0.535463,0.504923,0.501216,0.497338,0.511560
saccade_duration_maximum,0.498133,0.502942,0.510091,0.499866,0.516163,0.505439
saccade_duration_range,0.497974,0.503485,0.509276,0.499778,0.514863,0.505075


In [22]:
corr_scores = calculate_feature_correlation_scores(df_features_outliers_removed, df_label)
corr_scores

fixation_duration_median                     0.436589
fixation_duration_mean                       0.450048
fixation_duration_standard_deviation         0.511548
fixation_duration_minimum                    0.272391
fixation_duration_maximum                    0.522801
fixation_duration_range                      0.519196
fixation_duration_skew                       0.447909
fixation_duration_kurtosis                   0.442415
saccade_duration_median                      0.349130
saccade_duration_mean                        0.474160
saccade_duration_standard_deviation          0.492937
saccade_duration_minimum                     0.361990
saccade_duration_maximum                     0.501624
saccade_duration_range                       0.501344
saccade_duration_skew                        0.481916
saccade_duration_kurtosis                    0.482247
saccade_amplitude_median                     0.409661
saccade_amplitude_mean                       0.427816
saccade_amplitude_standard_d

In [23]:
label_corr = df_features.corrwith(df_label).fillna(0)
label_corr = label_corr.to_numpy()
label_corr = np.absolute(label_corr)
label_corr

array([5.33903924e-02, 7.72391365e-02, 6.06034617e-02, 5.45391320e-02,
       5.92537779e-02, 5.44889377e-02, 1.95031141e-02, 8.40191938e-03,
       2.86185125e-02, 1.10180361e-02, 1.07848817e-03, 9.73071620e-04,
       6.34351762e-03, 4.37205212e-03, 1.04284976e-02, 1.12130221e-02,
       5.64466115e-02, 9.24638328e-02, 1.16507373e-01, 4.59094267e-03,
       1.23734863e-01, 1.24261232e-01, 3.80201719e-02, 1.07630561e-02,
       2.79750054e-03, 6.57091705e-03, 3.40881651e-02, 3.80847835e-02,
       1.71056876e-02, 3.78667829e-02, 1.07087053e-02, 4.25688664e-03,
       4.08849549e-02, 4.00435807e-02, 9.84588932e-06, 1.47154650e-03,
       1.75942409e-02, 3.35555807e-03, 3.28045882e-02, 4.05487522e-03,
       2.23434718e-02, 3.78104800e-02, 5.38110979e-02, 7.80113298e-03,
       4.84405080e-02, 4.68348838e-02, 3.45369673e-02, 2.93748203e-03,
       1.51252804e-01, 1.50154545e-01, 1.93228668e-02, 1.19302681e-01,
       1.27458014e-01, 2.01011900e-02, 3.92972162e-02, 4.12209472e-02,
      

# Decision Tree Based Feature Selection

A more standard ML approach to feature selection is to build decision trees and use the level where features appear first
for decisions to be a proxy for their importance.  We will do that here, using the general method of selection
a 66% sample of subjects, determing feature rankings, and then averaging over 5 random selections and feature
importance determinations.

## Unscaled Feature Selection

In [24]:
# imports needed for tree based feature selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

Lets first try a simple tree-based feature selection, direct from scikit-learn examples in their
feature selection documentation.

In [25]:
clf = ExtraTreesClassifier(n_estimators=500)
clf = clf.fit(df_features, df_label)
clf.feature_importances_

array([0.01578196, 0.01543234, 0.01526021, 0.01639731, 0.01474488,
       0.01521782, 0.01518193, 0.01563294, 0.01476192, 0.01417772,
       0.01376058, 0.0105718 , 0.01373838, 0.01386066, 0.0152972 ,
       0.01573839, 0.0161176 , 0.0157149 , 0.01882917, 0.01684461,
       0.02032961, 0.0211794 , 0.01557152, 0.01558788, 0.01493665,
       0.01660774, 0.01498979, 0.01659946, 0.01515348, 0.01544423,
       0.01487142, 0.01528947, 0.01585028, 0.01498611, 0.01517906,
       0.01425597, 0.01514587, 0.01437095, 0.01525631, 0.0152128 ,
       0.0155431 , 0.01484707, 0.01702053, 0.01456311, 0.01500331,
       0.01447331, 0.01470331, 0.01545851, 0.02805566, 0.02863621,
       0.01615984, 0.02310804, 0.02269367, 0.0165898 , 0.01705379,
       0.01620276, 0.01243051, 0.01312675, 0.01581925, 0.01610586,
       0.01602293, 0.01650234])

In [26]:
tree_scores = pd.Series(data=clf.feature_importances_, index=df_features.columns)
tree_scores = tree_scores.sort_values(ascending=False)
tree_scores.iloc[:10]

pupil_diameter_mean                          0.028636
pupil_diameter_median                        0.028056
pupil_diameter_minimum                       0.023108
pupil_diameter_maximum                       0.022694
saccade_amplitude_range                      0.021179
saccade_amplitude_maximum                    0.020330
saccade_amplitude_standard_deviation         0.018829
pupil_diameter_skew                          0.017054
saccade_angle_relative_standard_deviation    0.017021
saccade_amplitude_minimum                    0.016845
dtype: float64

There is quite a lot of overlap, so I guess encouraging in sense the hand-made method seems reasonable.  But
seem like both of these are generating similar results, and lots of features I would have thought would be
correlated end up on top 10.

## Standard Scaled Feature Selection

As before, just to double check, lets see if we get about the same set of features and importance scores if we
use the scaled features

In [27]:
clf = ExtraTreesClassifier(n_estimators=500)
clf = clf.fit(df_features_standard_scaled, df_label)
clf.feature_importances_

array([0.01592795, 0.01544608, 0.01532795, 0.01602912, 0.01466867,
       0.01534414, 0.0152092 , 0.0156781 , 0.0147308 , 0.01401781,
       0.01371852, 0.01037758, 0.01416188, 0.01410618, 0.01517885,
       0.01564859, 0.01577498, 0.01590184, 0.01820968, 0.01678862,
       0.02080988, 0.02135262, 0.01519833, 0.01510202, 0.0151895 ,
       0.01626731, 0.01510224, 0.01636544, 0.01539407, 0.01551939,
       0.01511554, 0.01500946, 0.01571997, 0.01555516, 0.0151694 ,
       0.01438095, 0.01526187, 0.01421271, 0.01505836, 0.01509262,
       0.01526939, 0.01492581, 0.01700931, 0.01496835, 0.01560529,
       0.01487044, 0.01478866, 0.01484135, 0.02862818, 0.02816359,
       0.01601118, 0.02352217, 0.0224045 , 0.01676446, 0.01725883,
       0.01623243, 0.01237685, 0.01333857, 0.01569252, 0.01561653,
       0.01580464, 0.01678358])

In [28]:
tree_scores = pd.Series(data=clf.feature_importances_, index=df_features.columns)
tree_scores = tree_scores.sort_values(ascending=False)
tree_scores.iloc[:10]

pupil_diameter_median                        0.028628
pupil_diameter_mean                          0.028164
pupil_diameter_minimum                       0.023522
pupil_diameter_maximum                       0.022405
saccade_amplitude_range                      0.021353
saccade_amplitude_maximum                    0.020810
saccade_amplitude_standard_deviation         0.018210
pupil_diameter_skew                          0.017259
saccade_angle_relative_standard_deviation    0.017009
saccade_amplitude_minimum                    0.016789
dtype: float64

Maybe a little more variation than with previous method.  First 2 features were flipped in order.  But again here the
trees/forest have some randomness in the classifiers created, so we would expect differences.  

# Recursive Feature Elimination

Another method for feature selection/evaluation standard now in scikit learn.  Uses an estimator to rank features,
eliminate the wors one, and do this recursively untill it pairs down to the number of features you ask to select.

Not sure if a SVC (a support vector classifier)? is a good estimator to user here.

In [29]:
# imports needed for RFE feature selection
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

In [30]:
svc = SVC(kernel='linear', C=1)
rfe = RFE(estimator=svc, n_features_to_select=10, step=1)
# rfe.fit(df_features, df_label)  This takes some time, commented out so can rerun whole notebook without it

In [31]:
# the ranking_ attribute contains the iteration where feature dropped out.  for n=10 features, all features with a
# 1 ranking are in top 10
# maybe would be better to do the full selection, till n=1 feature, then we would have the ranking from 1 to 62 of
# when the feature dropped out, and thus could determine top 25%, top 50%, etc.
#print(rfe.ranking_)

# example of using this ranking to get the list of selected features.
#idx = (rfe.ranking_ == 1)
#rfe_features = df_features.columns[idx]
#print(rfe_features)

We capture the previous output in this text cell because of the time needed to rerun the rfe.

```
>>> idx = (rfe.ranking_ == 1)
[51 30 31 35 32 52 19 26 46 36 38 40 45 47  1 18 34 44 42 48 43 29  5 37
 27 12  7 13 20  9 11  6 50 28 33 24 22 23  1 25 49 41 21  1  4  3 14 15
  8  1  1  1  1 17 39  2 16 53 10  1  1  1]

>>> rfe_features = df_features.columns[idx]
>>> print(rfe_features)
Index(['saccade_duration_skew', 'saccade_angle_absolute_skew',
       'saccade_angle_relative_minimum', 'pupil_diameter_mean',
       'pupil_diameter_standard_deviation', 'pupil_diameter_minimum',
       'pupil_diameter_maximum', 'horizontal_saccade_proportion',
       'fixation_dispersion', 'fixation_saccade_durtion_ratio'],
      dtype='object')
      
```


The RFE feature selection is also identifying pupil diamater and some saccade measures.  The saccade measures
in previous two methods were mostly saccade_amplitude_X measures, where with RFE we get some different one.

horizontal_saccade_proportion did make one of the other top 10.  fixation_dispersion had not been seen before.

But tentatively I would conclude that it appears there is going to be a lot of overlap/similarity in
the feature selection ranking between RFE and others shown.  And because RFE seems much more computationally
intensive, we can probably use an alternative unless we find some reason to believe RFE may actually be
able to give better rankings that would improve modeling.

# VIF Feature Removal

I had initially skipped over this, but in the reference paper they actually were only using 32 of the features out of the
62 (though note, they say they have 62 features, and they say here (Machine learning to build the model, pg. 141)
that they removed 29 features.  This should leave 33 features, but they say the result of this feature removal
was 32 features, implying 30 were removed.  We will try this method and assume that 32 total features for model
building is the correct number, and try and remove 30.

The paper removed features with a variance inflation factor greater than 5 (i.e. R^2_i > 0.80).  I'm not immediately
familiar with this measure, but if using R^2 must be related to a linear regression fit.

Doesn't seem to have a VIF in scikit-learn or pandas (though I may have missed), but we can use
statsmodel function, using [this](https://www.statology.org/how-to-calculate-vif-in-python/)
as a reference.

In [32]:
# imports needed for VIF calculation
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [33]:
df = pd.DataFrame({'rating': [90, 85, 82, 88, 94, 90, 76, 75, 87, 86],
                   'points': [25, 20, 14, 16, 27, 20, 12, 15, 14, 19],
                   'assists': [5, 7, 7, 8, 5, 7, 6, 9, 9, 5],
                   'rebounds': [11, 8, 10, 6, 6, 9, 6, 10, 10, 7]})

df

,rating,points,assists,rebounds
0,90,25,5,11
1,85,20,7,8
2,82,14,7,10
3,88,16,8,6
4,94,27,5,6
5,90,20,7,9
6,76,12,6,6
7,75,15,9,10
8,87,14,9,10
9,86,19,5,7


In [34]:
#find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('rating ~ points+assists+rebounds', data=df, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

# show results
vif

,VIF,variable
0,101.258171,Intercept
1,1.763977,points
2,1.959104,assists
3,1.175030,rebounds


This replicates the result shown in the reference for calculating VIF.

This example uses dmatrices from the patsey library to create desing matrix for a linear regression, using
R like $\text{dependent} \sim \text{independent}_1 + \text{independent}_2 \cdots$ syntax.  We will want
to just construct that directly.  Actually the method is really simple, it returns a series with just rating
for the dependent vector and a dataframe with a column of 1 values to represent the intercept term
for linear regression.

Lest write a function to calcualte and return the same VIF score, though We'll use the variable label as
the index in a returned series, and drop the intercept VIF score.

In [35]:
def calculate_variance_inflation_factor(df_features):
    """Given a set of features, calculate the
    variance inflation factor (VIF) of the features given
    in the df_features DataFrame.
    
    We use [reference](https://www.statology.org/how-to-calculate-vif-in-python/)
    as source of the calculation, and actually this method really
    is just a wraper around the statsmodel implementation
    of VIF.
    """
    # construct the linear regression design matrices
    # make a copy of the features dataframe and add an intercept column feature as
    # first column, filled with 1'2
    num_trials, num_features = df_features.shape
    df = df_features.copy()
    df.insert( loc=0, column='intercept', value=np.ones(num_trials) )
    features_matrix = df.values
    
    # calculate the variance inflaction factor
    vif_results = [variance_inflation_factor(features_matrix, i) for i in range(num_features + 1)]
    vif = pd.Series(index = df.columns, data=vif_results)
    
    # we aren't interested in the intercept inflation factor for this feature trimming function,
    # so remove it
    vif = vif.iloc[1:]
    
    return vif

In [36]:
# we assume in our function that the label is already separate from the features, so remove the rating 
# label column first
df = pd.DataFrame({
    'points': [25, 20, 14, 16, 27, 20, 12, 15, 14, 19],               
    'assists': [5, 7, 7, 8, 5, 7, 6, 9, 9, 5],
    'rebounds': [11, 8, 10, 6, 6, 9, 6, 10, 10, 7]
})

# test on the example basketball dataframe again, should get the same result
calculate_variance_inflation_factor(df)

points      1.763977
assists     1.959104
rebounds    1.175030
dtype: float64

Ok with this function, we can ask for the VIF scores of our features.  The raw VIF score here I assume relates
to the $R^2$ fit.  When they say they removed variables with a VIF factor greater than 5, I assume any VIF greater than
5 we see calculated should be removed.

Lets see how many features have $\text{VIF} > 5$ in our 62 features (and in the scaled features, though the VIF should
be identical here).

In [37]:
vif = calculate_variance_inflation_factor(df_features)
vif

fixation_duration_median                     8.973671e+00
fixation_duration_mean                       3.197700e+01
fixation_duration_standard_deviation         6.511605e+01
fixation_duration_minimum                    6.017175e+00
fixation_duration_maximum                    1.355609e+02
fixation_duration_range                      6.688718e+01
fixation_duration_skew                       7.677336e+00
fixation_duration_kurtosis                   8.353964e+00
saccade_duration_median                      5.605104e+00
saccade_duration_mean                        4.710402e+01
saccade_duration_standard_deviation          1.414381e+02
saccade_duration_minimum                     2.183605e+00
saccade_duration_maximum                     4.897241e+01
saccade_duration_range                       1.263064e+02
saccade_duration_skew                        1.117154e+01
saccade_duration_kurtosis                    1.138122e+01
saccade_amplitude_median                     6.686922e+00
saccade_amplit

In [38]:
sum(vif > 5.0)

50

In [39]:
sum(vif > 16.0)

30

So I may have a bit of a descrepancy here, I seem to have 50 features with $VIF > 5.0$.  A little experimenting shows
raising the VIF threshold to 16 shows 30 are above this threshold, leaving 32 below.

The reference article did not list the 32 features that remained after removal for large vif.  However, the Table 1
(pg. 143) does have a list of 32 features (with calculated effect sizes), so I guess these were the actual 32 features they
used.  Lets compare our list of features when we screen for vif to get 32 features with this list for
differences.


In [40]:
# using the threshold that results in 32 features, find out which ones we would keep with this
# criteria
idx = (vif < 16.0)
vif_features = df_features.columns[idx]
print (len(vif_features))

for i, feature_name in enumerate(vif_features):
    print("%02d: %s" % (i+1, feature_name) )

32
01: fixation_duration_median
02: fixation_duration_minimum
03: fixation_duration_skew
04: fixation_duration_kurtosis
05: saccade_duration_median
06: saccade_duration_minimum
07: saccade_duration_skew
08: saccade_duration_kurtosis
09: saccade_amplitude_median
10: saccade_amplitude_minimum
11: saccade_amplitude_skew
12: saccade_amplitude_kurtosis
13: saccade_velocity_sd
14: saccade_velocity_skew
15: saccade_velocity_kurtosis
16: saccade_angle_absolute_median
17: saccade_angle_absolute_standard_deviation
18: saccade_angle_absolute_maximum
19: saccade_angle_absolute_kurtosis
20: saccade_angle_relative_median
21: saccade_angle_relative_standard_deviation
22: saccade_angle_relative_minimum
23: saccade_angle_relative_kurtosis
24: pupil_diameter_standard_deviation
25: pupil_diameter_skew
26: pupil_diameter_kurtosis
27: number_of_blinks
28: blink_duration_mean
29: number_of_saccades
30: horizontal_saccade_proportion
31: fixation_dispersion
32: fixation_saccade_durtion_ratio


Checking them off by hand:

| Paper Feature Name/Label         | Cleaned dataset column name |
|----------------------------------|-----------------------------|
| Horizontal saccade proportion,   | 30: horizontal_saccade_proportion |
| Fixation duration median,        | 01: fixation_duration_median |
| Pupil diameter skew,             | 25: pupil_diameter_skew |
| Number of saccades,              | 29: number_of_saccades |
| Relative saccade angle range     | |
| Pupil diameter median            | |
| Absolute saccade angle mean      | |
| Saccade amplitude kurtosis,      | 12: saccade_amplitude_kurtosis |
| Fixation duration range          | |
| Relative saccade angle kurtosis, | 23: saccade_angle_relative_kurtosis |
| Relative saccade angle median,   | 20: saccade_angle_relative_median |
| Relative saccade angle max       | |
| Absolute saccade angle SD,       | 17: saccade_angle_absolute_standard_deviation |
| Absolute saccade angle max,      | 18: saccade_angle_absolute_maximum |
| Saccade amplitude SD             | |
| Relative saccade angle skew      | |
| Saccade duration max             | |
| Absolute saccade angle mean      | |
| Saccade duration kurtosis,       | 08: saccade_duration_kurtosis |
| Saccade amplitude median,        | 09: saccade_amplitude_median |
| Absolute saccade angle median,   | 16: saccade_angle_absolute_median |
| Fixation dispersion mean,        | 31: fixation_dispersion (note this was named FxDisp in raw data, this is root mean square..) |
| Pupil diameter SD,               | 24: pupil_diameter_standard_deviation |
| Fixation duration kurtosis,      | 04: fixation_duration_kurtosis |
| Saccade velocity SD,             | 13: saccade_velocity_sd |
| Fixation/saccade ratio,          | 32: fixation_saccade_durtion_ratio |
| Pupil diameter kurtosis,         | 26: pupil_diameter_kurtosis |
| Absolute saccade angle kurtosis, | 19: saccade_angle_absolute_kurtosis |
| Saccade velocity skew,           | 14: saccade_velocity_skew |
| Saccade velocity kurtosis,       | 15: saccade_velocity_kurtosis |
| Saccade duration median,         | 05: saccade_duration_median |
| Number of blinks,                | 27: number_of_blinks |

Not in paper:

- 02: fixation_duration_minimum
- 03: fixation_duration_skew
- 06: saccade_duration_minimum
- 07: saccade_duration_skew
- 10: saccade_amplitude_minimum
- 11: saccade_amplitude_skew
- 21: saccade_angle_relative_standard_deviation
- 22: saccade_angle_relative_minimum
- 28: blink_duration_mean

By my count there are 9 in our vif set not in the paper, and vice versa.  Not sure if this is a real issue or not.
If it was 2 or 3 I was prepared to chalk it up to some differences in the datasets.  For example, I know that the
number of participants is not exactly the same, thus we have 3 more participants data in our starting raw data than
seem to have been reported in the paper.  This can certainly cause some differences.  But 9 out of 32 here, and the
seeming difference in the vif score threshold makes me a bit unsure.

I will probably proceed using the 32 that our vif threshold calculation selects as a subset.  Though another 
approch might be to reduce the threshold a bit until we get the full 32 shown in the paper, plus whatever extras
we need to get correct vif threshold to get these 32 features.

But just to document this, lets see what threshold we need to reinstate the features we don't have from the paper, and how
many features that ends up giving us in total.

This might not be the complete minimal result, but I think I had to go to the threshold shown below before I got back
all of the missing features.  At this threshold we end up with 50 making the cut and only eliminating 12.

In [41]:
# using the threshold that results in 32 features, find out which ones we would keep with this
# criteria
idx = (vif < 85.0)
vif_features = df_features.columns[idx]
print (len(vif_features))

for i, feature_name in enumerate(vif_features):
    print("%02d: %s" % (i+1, feature_name) )

50
01: fixation_duration_median
02: fixation_duration_mean
03: fixation_duration_standard_deviation
04: fixation_duration_minimum
05: fixation_duration_range
06: fixation_duration_skew
07: fixation_duration_kurtosis
08: saccade_duration_median
09: saccade_duration_mean
10: saccade_duration_minimum
11: saccade_duration_maximum
12: saccade_duration_skew
13: saccade_duration_kurtosis
14: saccade_amplitude_median
15: saccade_amplitude_mean
16: saccade_amplitude_standard_deviation
17: saccade_amplitude_minimum
18: saccade_amplitude_skew
19: saccade_amplitude_kurtosis
20: saccade_velocity_median
21: saccade_velocity_mean
22: saccade_velocity_sd
23: saccade_velocity_skew
24: saccade_velocity_kurtosis
25: saccade_angle_absolute_median
26: saccade_angle_absolute_mean
27: saccade_angle_absolute_standard_deviation
28: saccade_angle_absolute_minimum
29: saccade_angle_absolute_maximum
30: saccade_angle_absolute_range
31: saccade_angle_absolute_skew
32: saccade_angle_absolute_kurtosis
33: saccade_an

## Useing Feature Scaling and after Removing Outliers

It may be the case that VIF thresholding is more sensitive to outliers, and maybe scaling, then I had thought.  Lets try the vif threshold
again, but using the scaled and Winsorized data.

In [42]:
# get the features dataframe standard scaled and with outliers removed
df_features = mindwandering.data.transform_df_features_outliers_removed(df_features_standard_scaled)

In [43]:
vif = calculate_variance_inflation_factor(df_features)
vif

fixation_duration_median                        7.537594
fixation_duration_mean                         27.011213
fixation_duration_standard_deviation           57.575436
fixation_duration_minimum                       9.502688
fixation_duration_maximum                     287.448349
fixation_duration_range                       231.912763
fixation_duration_skew                          8.476941
fixation_duration_kurtosis                      8.377739
saccade_duration_median                         5.430114
saccade_duration_mean                          41.292158
saccade_duration_standard_deviation           156.767704
saccade_duration_minimum                        2.950171
saccade_duration_maximum                      132.062517
saccade_duration_range                        248.287031
saccade_duration_skew                          15.132721
saccade_duration_kurtosis                      15.503195
saccade_amplitude_median                        7.015915
saccade_amplitude_mean         

In [44]:
sum(vif > 17.0)

30

Now we have a  issue in the other direction.  After standard scaling and removing outliers, vif scores are much lower.

Lets find threshold that gives us 32 features selected again, and look at the features that pass the threshold now.

In [45]:
sum(vif > 17.0)

30

In [46]:
idx = (vif < 17.0)
vif_features = df_features.columns[idx]
print (len(vif_features))

for i, feature_name in enumerate(vif_features):
    print("%02d: %s" % (i+1, feature_name) )

32
01: fixation_duration_median
02: fixation_duration_minimum
03: fixation_duration_skew
04: fixation_duration_kurtosis
05: saccade_duration_median
06: saccade_duration_minimum
07: saccade_duration_skew
08: saccade_duration_kurtosis
09: saccade_amplitude_median
10: saccade_amplitude_minimum
11: saccade_amplitude_skew
12: saccade_velocity_sd
13: saccade_velocity_skew
14: saccade_velocity_kurtosis
15: saccade_angle_absolute_median
16: saccade_angle_absolute_standard_deviation
17: saccade_angle_absolute_maximum
18: saccade_angle_absolute_kurtosis
19: saccade_angle_relative_median
20: saccade_angle_relative_standard_deviation
21: saccade_angle_relative_minimum
22: saccade_angle_relative_maximum
23: saccade_angle_relative_kurtosis
24: pupil_diameter_standard_deviation
25: pupil_diameter_skew
26: pupil_diameter_kurtosis
27: number_of_blinks
28: blink_duration_mean
29: number_of_saccades
30: horizontal_saccade_proportion
31: fixation_dispersion
32: fixation_saccade_durtion_ratio


I may need to revisit this issue.  These are about as different again from my vif on the unscaled data and from the features shows in the original paper.